# Start-to-Finish Example: Head-On Black Hole Collision

## Author: Zach Etienne
### Formatting improvements courtesy Brandon Clark

## This module implements a basic numerical relativity code to merge two black holes in *spherical-like coordinates*, as well as the gravitational wave analysis provided by the $\psi_4$ NRPy+ tutorial modules ([$\psi_4$](Tutorial-Psi4.ipynb) & [$\psi_4$ tetrad](Tutorial-Psi4_tetrads.ipynb)).

### Here we place the black holes initially on the $z$-axis, so the entire simulation is axisymmetric about the $\phi$-axis. Minimal sampling in the $\phi$ direction greatly speeds up the simulation.

**Module Status:** <font color = green><b> Validated </b></font>

**Validation Notes:** This module has been validated to exhibit convergence to zero of the Hamiltonian constraint violation at the expected order to the exact solution *after a short numerical evolution of the initial data* (see [plots at bottom](#convergence)), and all quantities have been validated against the [original SENR code](https://bitbucket.org/zach_etienne/nrpy). Finally, excellent agreement is seen in the gravitational wave signal from the ringing remnant black hole for multiple decades in amplitude when compared to black hole perturbation theory predictions.

### NRPy+ Source Code for this module: 
* [BSSN/BrillLindquist.py](../edit/BSSN/BrillLindquist.py); [\[**tutorial**\]](Tutorial-ADM_Initial_Data-Brill-Lindquist.ipynb): Brill-Lindquist initial data; sets all ADM variables in Cartesian basis: 
* [BSSN/ADM_Exact_Spherical_or_Cartesian_to_BSSNCurvilinear.py](../edit/BSSN/ADM_Exact_Spherical_or_Cartesian_to_BSSNCurvilinear.py); [\[**tutorial**\]](Tutorial-ADM_Initial_Data-Converting_Exact_ADM_Spherical_or_Cartesian_to_BSSNCurvilinear.ipynb): Spherical/Cartesian ADM$\to$Curvilinear BSSN converter function, for which exact expressions are given for ADM quantities.
* [BSSN/BSSN_ID_function_string.py](../edit/BSSN/BSSN_ID_function_string.py): Sets up the C code string enabling initial data be set up in a point-by-point fashion
* [BSSN/BSSN_constraints.py](../edit/BSSN/BSSN_constraints.py); [\[**tutorial**\]](Tutorial-BSSN_constraints.ipynb): Hamiltonian constraint in BSSN curvilinear basis/coordinates
* [BSSN/BSSN_RHSs.py](../edit/BSSN/BSSN_RHSs.py); [\[**tutorial**\]](Tutorial-BSSN_time_evolution-BSSN_RHSs.ipynb): Generates the right-hand sides for the BSSN evolution equations in singular, curvilinear coordinates
* [BSSN/BSSN_gauge_RHSs.py](../edit/BSSN/BSSN_gauge_RHSs.py); [\[**tutorial**\]](Tutorial-BSSN_time_evolution-BSSN_gauge_RHSs.ipynb): Generates the right-hand sides for the BSSN gauge evolution equations in singular, curvilinear coordinates
* [BSSN/Psi4.py](../edit/BSSN/Psi4.py); [\[**tutorial**\]](Tutorial-Psi4.ipynb): Generates expressions for $\psi_4$, the outgoing Weyl scalar
    + [BSSN/Psi4_tetrads.py](../edit/BSSN/Psi4_tetrads.py); [\[**tutorial**\]](Tutorial-Psi4_tetrads.ipynb): Generates quasi-Kinnersley tetrad needed for $\psi_4$-based gravitational wave extraction

## Introduction:
Here we use NRPy+ to generate the C source code necessary to set up initial data for two black holes (Brill-Lindquist, [Brill & Lindquist, Phys. Rev. 131, 471, 1963](https://journals.aps.org/pr/abstract/10.1103/PhysRev.131.471); see also Eq. 1 of [Brandt & Brügmann, arXiv:gr-qc/9711015v1](https://arxiv.org/pdf/gr-qc/9711015v1.pdf)). Then we use it to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

The entire algorithm is outlined below, with NRPy+-based components highlighted in <font color='green'>green</font>.

1. Allocate memory for gridfunctions, including temporary storage for the RK4 time integration.
1. <font color='green'>Set gridfunction values to initial data (**[documented in previous start-to-finish module](Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_Exact_Initial_Data.ipynb)**).</font>
1. Evolve the initial data forward in time using RK4 time integration. At each RK4 substep, do the following:
    1. <font color='green'>Evaluate BSSN RHS expressions.</font>
    1. Apply singular, curvilinear coordinate boundary conditions [*a la* the SENR/NRPy+ paper](https://arxiv.org/abs/1712.07658)
    1. <font color='green'>Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint</font>
1. At the end of each iteration in time, output the <font color='green'>Hamiltonian constraint violation</font>.
1. Repeat the above steps at two numerical resolutions to confirm convergence to zero.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#initializenrpy): Set core NRPy+ parameters for numerical grids and reference metric
1. [Step 2](#adm_id): Import Brill-Lindquist ADM initial data C function from the [`BSSN.BrillLindquist`](../edit/BSSN/BrillLindquist.py) NRPy+ module
1. [Step 3](#bssn): Output C code for BSSN spacetime solve
    1. [Step 3.a](#bssnrhs): Output C code for BSSN RHS expressions
    1. [Step 3.b](#hamconstraint): Output C code for Hamiltonian constraint
    1. [Step 3.c](#enforce3metric): Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint
    1. [Step 3.d](#spinweight): Computing $_{-2}Y_{\ell m} (\theta, \phi)$ for all $(\ell,m)$ for $\ell=0$ up to 2
    1. [Step 3.e](#psi4): $\psi_4$
    1. [Step 3.f](#cparams_rfm_and_domainsize): Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h`
1. [Step 4](#bc_functs): Set up boundary condition functions for chosen singular, curvilinear coordinate system
1. [Step 5](#mainc): `BrillLindquist_Playground.c`: The Main C Code
1. [Step 6](#visualize): Data Visualization Animations
    1. [Step 6.a](#installdownload): Install `scipy` and download `ffmpeg` if they are not yet installed/downloaded
    1. [Step 6.b](#genimages): Generate images for visualization animation
    1. [Step 6.c](#genvideo): Generate visualization animation
1. [Step 7](#convergence): Visualize the numerical error, and confirm that it converges to zero with increasing numerical resolution (sampling)
1. [Step 8](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Set core NRPy+ parameters for numerical grids and reference metric \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

In [1]:
# First we import needed core NRPy+ modules
from outputC import *
import NRPy_param_funcs as par
import grid as gri
import loop as lp
import indexedexp as ixp
import finite_difference as fin
import reference_metric as rfm
import cmdline_helper as cmd
import shutil, os

# Create directory for all C codes output by this module:
outdir = "BSSN_Two_BHs_Collide-Psi4_Ccodes/"
cmd.mkdir(outdir)

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 2: Set some core parameters, including CoordSystem MoL timestepping algorithm,
#                                 FD order, floating point precision, and CFL factor:
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical, 
#              SymTP, SinhSymTP
CoordSystem     = "SinhSpherical"
domain_size     = 300 # Needed for all coordinate systems.
sinh_width      = 0.2 # If Sinh* coordinates chosen
sinhv2_const_dr = 0.05# If Sinh*v2 coordinates chosen
SymTP_bScale    = 0.5 # If SymTP chosen
# Choices are: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
RK_method = "RK4"
FD_order  = 10       # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "double" # Best to use double here.
CFL_FACTOR= 0.5      # (GETS OVERWRITTEN WHEN EXECUTED.) In pure axisymmetry (symmetry_axes = 2 below) 1.0 works fine. Otherwise 0.5 or lower.

# Generate timestepping code. As described above the Table of Contents, this is a 3-step process:
#       3.A: Evaluate RHSs (RHS_string)
#       3.B: Apply boundary conditions (post_RHS_string, pt 1)
#       3.C: Enforce det(gammabar) = det(gammahat) constraint (post_RHS_string, pt 2)
import MoLtimestepping.C_Code_Generation as MoL
from MoLtimestepping.RK_Butcher_Table_Dictionary import Butcher_dict
RK_order  = Butcher_dict[RK_method][1]
cmd.mkdir(os.path.join(outdir,"MoLtimestepping/"))
MoL.MoL_C_Code_Generation(RK_method, 
    RHS_string      = """
Ricci_eval(&rfmstruct, &params, RK_INPUT_GFS, auxevol_gfs);
rhs_eval(&rfmstruct, &params, auxevol_gfs, RK_INPUT_GFS, RK_OUTPUT_GFS);""",
    post_RHS_string = """
apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_OUTPUT_GFS);
enforce_detgammabar_constraint(&rfmstruct, &params,                     RK_OUTPUT_GFS);\n""",
    outdir = os.path.join(outdir,"MoLtimestepping/"))

# Set finite differencing order:
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

# REAL and CFL_FACTOR parameters used below in C code directly

# Then we set the coordinate system for the numerical grid
# NOTE: Only CoordSystem == Spherical or SinhSpherical
#       currently work out of the box; additional modifications
#       will likely be necessary for other CoordSystems.
par.set_parval_from_str("reference_metric::CoordSystem",CoordSystem)
rfm.reference_metric() # Create ReU, ReDD needed for rescaling B-L initial data, generating BSSN RHSs, etc.

# Copy SIMD/SIMD_intrinsics.h to $outdir/SIMD/SIMD_intrinsics.h
cmd.mkdir(os.path.join(outdir,"SIMD"))
shutil.copy(os.path.join("SIMD/")+"SIMD_intrinsics.h",os.path.join(outdir,"SIMD/"))

# Then we set the phi axis to be the symmetry axis; i.e., axis "2", corresponding to the i2 direction. 
#      This sets all spatial derivatives in the phi direction to zero.
par.set_parval_from_str("indexedexp::symmetry_axes","2")

<a id='cfl'></a>

## Step 1.c: Output needed C code for finding the minimum proper distance between grid points, needed for [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673)-limited timestep \[Back to [top](#toc)\]
$$\label{cfl}$$

In order for our explicit-timestepping numerical solution to the scalar wave equation to be stable, it must satisfy the [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673) condition:
$$
\Delta t \le \frac{\min(ds_i)}{c},
$$
where $c$ is the wavespeed, and
$$ds_i = h_i \Delta x^i$$ 
is the proper distance between neighboring gridpoints in the $i$th direction (in 3D, there are 3 directions), $h_i$ is the $i$th reference metric scale factor, and $\Delta x^i$ is the uniform grid spacing in the $i$th direction:

In [2]:
# Generic coordinate NRPy+ file output, Part 4: output the proper distance between gridpoints in given coordinate system
dxx     = ixp.declarerank1("dxx",DIM=3)
ds_dirn = rfm.ds_dirn(dxx)
outputC([ds_dirn[0],ds_dirn[1],ds_dirn[2]],["ds_dirn0","ds_dirn1","ds_dirn2"],outdir+"ds_dirn.h")

Wrote to file "BSSN_Two_BHs_Collide-Psi4_Ccodes/ds_dirn.h"


<a id='adm_id'></a>

# Step 2: Import Brill-Lindquist ADM initial data C function from the [`BSSN.BrillLindquist`](../edit/BSSN/BrillLindquist.py) NRPy+ module \[Back to [top](#toc)\]
$$\label{adm_id}$$

The [`BSSN.BrillLindquist`](../edit/BSSN/BrillLindquist.py) NRPy+ module does the following:

1. Set up Brill-Lindquist initial data [ADM](https://en.wikipedia.org/wiki/ADM_formalism) quantities in the **Cartesian basis**, as [documented here](Tutorial-ADM_Initial_Data-Brill-Lindquist.ipynb). 
1. Convert the ADM **Cartesian quantities** to **BSSN quantities in the desired Curvilinear basis** (set by reference_metric::CoordSystem), as [documented here](Tutorial-ADM_Initial_Data-Converting_ADMCartesian_to_BSSNCurvilinear.ipynb).
1. Sets up the standardized C function for setting all BSSN Curvilinear gridfunctions in a pointwise fashion, as [written here](../edit/BSSN/BSSN_ID_function_string.py), and returns the C function as a Python string.

In [3]:
import time

import BSSN.BrillLindquist as bl
def BrillLindquistID():
    print("Generating optimized C code for Brill-Lindquist initial data. May take a while, depending on CoordSystem.")
    start = time.time()
    
    returnfunction = bl.BrillLindquist(returnfunctionversion=2)
    # Now output the Brill-Lindquist initial data to file:
    with open(outdir+"BrillLindquist.h","w") as file:
        file.write(bl.returnfunction)
    end = time.time()
    print("Finished BL initial data codegen in "+str(end-start)+" seconds.")

<a id='bssn'></a>

# Step 3: Output C code for BSSN spacetime solve \[Back to [top](#toc)\]
$$\label{bssn}$$

<a id='bssnrhs'></a>

## Step 3.a: Output C code for BSSN RHS expressions \[Back to [top](#toc)\]
$$\label{bssnrhs}$$

In [4]:
import BSSN.BSSN_RHSs as rhs
import BSSN.BSSN_gauge_RHSs as gaugerhs
# Set the *covariant*, second-order Gamma-driving shift condition
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::ShiftEvolutionOption", "GammaDriving2ndOrder_Covariant")

print("Generating symbolic expressions for BSSN RHSs...")
start = time.time()
# Enable rfm_precompute infrastructure, which results in 
#   BSSN RHSs that are free of transcendental functions,
#   even in curvilinear coordinates, so long as 
#   ConformalFactor is set to "W" (default).
cmd.mkdir(os.path.join(outdir,"rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute","True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir",os.path.join(outdir,"rfm_files/"))

# Evaluate BSSN + BSSN gauge RHSs with rfm_precompute enabled:
import BSSN.BSSN_quantities as Bq
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic","True")

rhs.BSSN_RHSs()
gaugerhs.BSSN_gauge_RHSs()

# We use betaU as our upwinding control vector:
Bq.BSSN_basic_tensors()
betaU = Bq.betaU

import BSSN.Enforce_Detgammabar_Constraint as EGC
enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammabar_Constraint_symb_expressions()

# Next compute Ricci tensor
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic","False")
Bq.RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()

# Now that we are finished with all the rfm hatted
#           quantities in generic precomputed functional
#           form, let's restore them to their closed-
#           form expressions.
par.set_parval_from_str("reference_metric::enable_rfm_precompute","False") # Reset to False to disable rfm_precompute.
rfm.ref_metric__hatted_quantities()
end = time.time()
print("Finished BSSN symbolic expressions in "+str(end-start)+" seconds.")

def BSSN_RHSs():
    print("Generating C code for BSSN RHSs in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()

    BSSN_evol_rhss = [ \
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD00"),rhs=rhs.a_rhsDD[0][0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD01"),rhs=rhs.a_rhsDD[0][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD02"),rhs=rhs.a_rhsDD[0][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD11"),rhs=rhs.a_rhsDD[1][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD12"),rhs=rhs.a_rhsDD[1][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","aDD22"),rhs=rhs.a_rhsDD[2][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","alpha"),rhs=gaugerhs.alpha_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU0"),rhs=gaugerhs.bet_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU1"),rhs=gaugerhs.bet_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","betU2"),rhs=gaugerhs.bet_rhsU[2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","cf"),   rhs=rhs.cf_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD00"),rhs=rhs.h_rhsDD[0][0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD01"),rhs=rhs.h_rhsDD[0][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD02"),rhs=rhs.h_rhsDD[0][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD11"),rhs=rhs.h_rhsDD[1][1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD12"),rhs=rhs.h_rhsDD[1][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","hDD22"),rhs=rhs.h_rhsDD[2][2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU0"),rhs=rhs.lambda_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU1"),rhs=rhs.lambda_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","lambdaU2"),rhs=rhs.lambda_rhsU[2]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","trK"),  rhs=rhs.trK_rhs),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU0"),rhs=gaugerhs.vet_rhsU[0]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU1"),rhs=gaugerhs.vet_rhsU[1]),
                      lhrh(lhs=gri.gfaccess("rhs_gfs","vetU2"),rhs=gaugerhs.vet_rhsU[2]) ]

    BSSN_RHSs_string = fin.FD_outputC("returnstring",BSSN_evol_rhss, params="outCverbose=False,SIMD_enable=True",
                                      upwindcontrolvec=betaU)

    with open(outdir+"BSSN_RHSs.h", "w") as file:
        file.write(lp.loop(["i2","i1","i0"],["NGHOSTS","NGHOSTS","NGHOSTS"],
                           ["NGHOSTS+Nxx2","NGHOSTS+Nxx1","NGHOSTS+Nxx0"],
                           ["1","1","SIMD_width"],
                            ["#pragma omp parallel for",
                                         "#include \"rfm_files/rfm_struct__SIMD_outer_read2.h\"",
                                         "#include \"rfm_files/rfm_struct__SIMD_outer_read1.h\""],"",
                                         "#include \"rfm_files/rfm_struct__SIMD_inner_read0.h\"\n"+BSSN_RHSs_string.replace("IDX4","IDX4S")))
    end = time.time()
    print("Finished BSSN_RHS C codegen in " + str(end - start) + " seconds.")
    
def Ricci():
    print("Generating C code for Ricci tensor in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()
    Ricci_string = fin.FD_outputC("returnstring",
                                  [lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD00"),rhs=Bq.RbarDD[0][0]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD01"),rhs=Bq.RbarDD[0][1]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD02"),rhs=Bq.RbarDD[0][2]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD11"),rhs=Bq.RbarDD[1][1]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD12"),rhs=Bq.RbarDD[1][2]),
                                   lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD22"),rhs=Bq.RbarDD[2][2])],
                                   params="outCverbose=False,SIMD_enable=True")
    with open(outdir+"BSSN_Ricci.h", "w") as file:
        file.write(lp.loop(["i2","i1","i0"],["NGHOSTS","NGHOSTS","NGHOSTS"],
                           ["NGHOSTS+Nxx2","NGHOSTS+Nxx1","NGHOSTS+Nxx0"],
                           ["1","1","SIMD_width"],
                            ["#pragma omp parallel for",
                                         "#include \"rfm_files/rfm_struct__SIMD_outer_read2.h\"",
                                         "#include \"rfm_files/rfm_struct__SIMD_outer_read1.h\""],"",
                                         "#include \"rfm_files/rfm_struct__SIMD_inner_read0.h\"\n"+Ricci_string.replace("IDX4","IDX4S")))
    end = time.time()
    print("Finished Ricci C codegen in " + str(end - start) + " seconds.")

Generating symbolic expressions for BSSN RHSs...
Finished BSSN symbolic expressions in 12.3932518959 seconds.


<a id='hamconstraint'></a>

## Step 3.b: Output C code for Hamiltonian constraint \[Back to [top](#toc)\]
$$\label{hamconstraint}$$

Next output the C code for evaluating the Hamiltonian constraint. In the absence of numerical error, this constraint should evaluate to zero. However it does not due to numerical (typically truncation and roundoff) error. We will therefore measure the Hamiltonian constraint violation to gauge the accuracy of our simulation, and, ultimately determine whether errors are dominated by numerical finite differencing (truncation) error as expected.

In [5]:
# First register the Hamiltonian as a gridfunction.
H = gri.register_gridfunctions("AUX","H")
# Then define the Hamiltonian constraint and output the optimized C code.
import BSSN.BSSN_constraints as bssncon

def Hamiltonian():
    bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)

    start = time.time()
    print("Generating optimized C code for Hamiltonian constraint. May take a while, depending on CoordSystem.")
    Hamiltonianstring = fin.FD_outputC("returnstring", lhrh(lhs=gri.gfaccess("aux_gfs", "H"), rhs=bssncon.H),
                                       params="outCverbose=False")

    with open(outdir+"Hamiltonian.h", "w") as file:
        file.write("""
    void Hamiltonian_constraint(const paramstruct *restrict params, REAL *restrict xx[3], 
                                REAL *restrict in_gfs, REAL *restrict aux_gfs) {
    #include "set_Cparameters.h"
    """)

        file.write(lp.loop(["i2","i1","i0"],["NGHOSTS","NGHOSTS","NGHOSTS"],
                           ["NGHOSTS+Nxx2","NGHOSTS+Nxx1","NGHOSTS+Nxx0"],
                           ["1","1","1"],
                           ["#pragma omp parallel for",
                            "    const REAL xx2 = xx[2][i2];",
                            "        const REAL xx1 = xx[1][i1];"], "",
                               "const REAL xx0 = xx[0][i0];\n" + Hamiltonianstring.replace("IDX4","IDX4S")))
        file.write("}\n")
    end = time.time()
    print("Finished Hamiltonian C codegen in " + str(end - start) + " seconds.")

<a id='enforce3metric'></a>

## Step 3.c: Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint \[Back to [top](#toc)\]
$$\label{enforce3metric}$$

Then enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (Eq. 53 of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)), as [documented in the corresponding NRPy+ tutorial module](Tutorial-BSSN-Enforcing_Determinant_gammabar_equals_gammahat_Constraint.ipynb)

Applying curvilinear boundary conditions should affect the initial data at the outer boundary, and will in general cause the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint to be violated there. Thus after we apply these boundary conditions, we must always call the routine for enforcing the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint:

In [6]:
def gammadet():
    start = time.time()
    print("Generating optimized C code for gamma constraint. May take a while, depending on CoordSystem.")
    enforce_gammadet_string = fin.FD_outputC("returnstring", enforce_detg_constraint_symb_expressions,
                                             params="outCverbose=False,preindent=0,includebraces=False")

    with open(outdir+"enforce_detgammabar_constraint.h", "w") as file:
        indent = "   "
        file.write("""
    void enforce_detgammabar_constraint(rfm_struct *restrict rfmstruct,
                                        const paramstruct *restrict params, REAL *restrict in_gfs) {
    #include "set_Cparameters.h"
    """)
        file.write(lp.loop(["i2","i1","i0"],["0", "0", "0"],
                           ["Nxx_plus_2NGHOSTS2", "Nxx_plus_2NGHOSTS1", "Nxx_plus_2NGHOSTS0"],
                           ["1","1","1"],
                            ["#pragma omp parallel for",
                                         "#include \"rfm_files/rfm_struct__read2.h\"",
                                         "#include \"rfm_files/rfm_struct__read1.h\""],"",
                                         "#include \"rfm_files/rfm_struct__read0.h\"\n"+enforce_gammadet_string.replace("IDX4","IDX4S")))
        file.write("}\n")
    end = time.time()
    print("Finished gamma constraint C codegen in " + str(end - start) + " seconds.")

<a id='spinweight'></a>

## Step 3.d: Computing $_{-2}Y_{\ell m} (\theta, \phi)$ for all $(\ell,m)$ for $\ell=0$ up to 2 \[Back to [top](#toc)\]
$$\label{spinweight}$$ 

[**Tutorial Module**](Tutorial-SpinWeighted_Spherical_Harmonics.ipynb)

In [7]:
import SpinWeight_minus2_SphHarmonics.SpinWeight_minus2_SphHarmonics as swm2
cmd.mkdir(outdir+"SpinWeight_minus2_SphHarmonics")
swm2.SpinWeight_minus2_SphHarmonics(maximum_l=2,filename=outdir+"SpinWeight_minus2_SphHarmonics/SpinWeight_minus2_SphHarmonics.h")

<a id='psi4'></a>

## Step 3.e: Output $\psi_4$ \[Back to [top](#toc)\]
$$\label{psi4}$$

We output $\psi_4$, assuming Quasi-Kinnersley tetrad of [Baker, Campanelli, Lousto (2001)](https://arxiv.org/pdf/gr-qc/0104063.pdf).

In [8]:
import BSSN.Psi4_tetrads as BP4t
par.set_parval_from_str("BSSN.Psi4_tetrads::TetradChoice","QuasiKinnersley")
#par.set_parval_from_str("BSSN.Psi4_tetrads::UseCorrectUnitNormal","True")
import BSSN.Psi4 as BP4
BP4.Psi4()

psi4r_0pt = gri.register_gridfunctions("AUX","psi4r_0pt")
psi4r_1pt = gri.register_gridfunctions("AUX","psi4r_1pt")
psi4r_2pt = gri.register_gridfunctions("AUX","psi4r_2pt")
psi4i_0pt = gri.register_gridfunctions("AUX","psi4i_0pt")
psi4i_1pt = gri.register_gridfunctions("AUX","psi4i_1pt")
psi4i_2pt = gri.register_gridfunctions("AUX","psi4i_2pt")

def Psi4re(part):
    print("Generating C code for psi4_re_pt"+str(part)+" in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()
    Psi4re_pt = fin.FD_outputC("returnstring",
                   [lhrh(lhs=gri.gfaccess("aux_gfs","psi4r_"+str(part)+"pt"),rhs=BP4.psi4_re_pt[part])],
                   params="outCverbose=False")
    with open(outdir+"Psi4re_pt"+str(part)+"_lowlevel.h", "w") as file:
        file.write(Psi4re_pt.replace("IDX4","IDX4S"))
    end = time.time()
    print("Finished generating psi4_re_pt"+str(part)+" in "+str(end-start)+" seconds.")
    
def Psi4im(part):
    print("Generating C code for psi4_im_pt"+str(part)+" in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()
    Psi4im_pt = fin.FD_outputC("returnstring",
                   [lhrh(lhs=gri.gfaccess("aux_gfs","psi4i_"+str(part)+"pt"),rhs=BP4.psi4_im_pt[part])],
                   params="outCverbose=False")
    with open(outdir+"Psi4im_pt"+str(part)+"_lowlevel.h", "w") as file:
        file.write(Psi4im_pt.replace("IDX4","IDX4S"))
    end = time.time()
    print("Finished generating psi4_im_pt"+str(part)+" in "+str(end-start)+" seconds.")

In [9]:
import multiprocessing

if __name__ == '__main__':
    ID  = multiprocessing.Process(target=BrillLindquistID)
    RHS = multiprocessing.Process(target=BSSN_RHSs)
    Ric = multiprocessing.Process(target=Ricci)
    Ham = multiprocessing.Process(target=Hamiltonian)
    gam = multiprocessing.Process(target=gammadet)
    Psi4re0 = multiprocessing.Process(target=Psi4re, args=(0,))
    Psi4re1 = multiprocessing.Process(target=Psi4re, args=(1,))
    Psi4re2 = multiprocessing.Process(target=Psi4re, args=(2,))
    Psi4im0 = multiprocessing.Process(target=Psi4im, args=(0,))
    Psi4im1 = multiprocessing.Process(target=Psi4im, args=(1,))
    Psi4im2 = multiprocessing.Process(target=Psi4im, args=(2,))
    
    ID.start()
    RHS.start()
    Ric.start()
    Ham.start()
    gam.start()
    Psi4re0.start()
    Psi4re1.start()
    Psi4re2.start()
    Psi4im0.start()
    Psi4im1.start()
    Psi4im2.start()
     
    ID.join()
    RHS.join()
    Ric.join()
    Ham.join()
    gam.join()
    Psi4re0.join()
    Psi4re1.join()
    Psi4re2.join()
    Psi4im0.join()
    Psi4im1.join()
    Psi4im2.join()

Generating optimized C code for Brill-Lindquist initial data. May take a while, depending on CoordSystem.
Generating C code for BSSN RHSs in SinhSpherical coordinates.
Generating C code for Ricci tensor in SinhSpherical coordinates.
Generating optimized C code for gamma constraint. May take a while, depending on CoordSystem.
Generating C code for psi4_re_pt0 in SinhSpherical coordinates.
Generating C code for psi4_re_pt1 in SinhSpherical coordinates.
Generating C code for psi4_re_pt2 in SinhSpherical coordinates.
Generating C code for psi4_im_pt0 in SinhSpherical coordinates.
Generating C code for psi4_im_pt1 in SinhSpherical coordinates.
Generating C code for psi4_im_pt2 in SinhSpherical coordinates.
Finished gamma constraint C codegen in 1.79141402245 seconds.
Finished generating psi4_im_pt1 in 12.6269309521 seconds.
Generating optimized C code for Hamiltonian constraint. May take a while, depending on CoordSystem.
Finished generating psi4_re_pt1 in 16.4502251148 seconds.
Finished ge

<a id='cparams_rfm_and_domainsize'></a>

## Step 3.f: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]
$$\label{cparams_rfm_and_domainsize}$$

Based on declared NRPy+ Cparameters, first we generate `declare_Cparameters_struct.h`, `set_Cparameters_default.h`, and `set_Cparameters[-SIMD].h`.

Then we output `free_parameters.h`, which sets initial data parameters, as well as grid domain & reference metric parameters, applying `domain_size` and `sinh_width`/`SymTP_bScale` (if applicable) as set above

In [10]:
# Step 3.f.i: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(outdir))

# Step 3.f.ii: Set free_parameters.h
with open(os.path.join(outdir,"free_parameters.h"),"w") as file:
    file.write("""
// Set free-parameter values.

// Set free-parameter values for BSSN evolution:
params.eta = 2.0;

// Set free parameters for the (Brill-Lindquist) initial data
params.BH1_posn_x = 0.0; params.BH1_posn_y = 0.0; params.BH1_posn_z =+0.25;
params.BH2_posn_x = 0.0; params.BH2_posn_y = 0.0; params.BH2_posn_z =-0.25;
params.BH1_mass = 0.5; params.BH2_mass = 0.5;

const REAL domain_size    = """+str(domain_size)+""";
const REAL sinh_width     = """+str(sinh_width)+""";
const REAL sinhv2_const_dr= """+str(sinhv2_const_dr)+""";
const REAL SymTP_bScale   = """+str(SymTP_bScale)+";\n")

    coordparams = ""
    if CoordSystem == "Spherical":
        coordparams += """
        params.RMAX = domain_size;\n"""
    elif "SinhSpherical" in CoordSystem:
        coordparams += """
        params.AMPL = domain_size;
        params.SINHW=  sinh_width;\n"""
        if CoordSystem == "SinhSphericalv2":
            coordparams += "        params.const_dr = sinhv2_const_dr;\n"
    elif "SymTP" in CoordSystem:
        coordparams += """
        params.bScale =  SymTP_bScale;
        params.AMAX   =  domain_size;\n"""
        if CoordSystem == "SinhSymTP":
            coordparams += "        params.SINHWAA = sinh_width;\n"
    elif CoordSystem == "Cartesian":
        coordparams += """
        params.xmin = -domain_size, params.xmax = domain_size;
        params.ymin = -domain_size, params.ymax = domain_size;
        params.zmin = -domain_size, params.zmax = domain_size;\n"""
    elif CoordSystem == "Cylindrical":
        coordparams += """
        params.ZMIN   = -domain_size;
        params.ZMAX   =  domain_size;
        params.RHOMAX =  domain_size;\n"""
    elif CoordSystem == "SinhCylindrical":
        coordparams += """
        params.AMPLRHO = domain_size;
        params.SINHWRHO= sinh_width;
        params.AMPLZ   = domain_size;
        params.SINHWZ  = sinh_width;\n"""
    file.write(coordparams+"\n")

# Step 3.f.iii: Generate set_Nxx_dxx_invdx_params__and__xx.h:
rfm.set_Nxx_dxx_invdx_params__and__xx_h(outdir)

# Step 3.f.iv: Generate xxCart.h, which contains xxCart() for
#               (the mapping from xx->Cartesian) for the chosen
#               CoordSystem:
rfm.xxCart_h("xxCart","./set_Cparameters.h",os.path.join(outdir,"xxCart.h"))

# Step 3.f.v: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(outdir))

<a id='bc_functs'></a>

# Step 4: Set up boundary condition functions for chosen singular, curvilinear coordinate system \[Back to [top](#toc)\]
$$\label{bc_functs}$$

Next apply singular, curvilinear coordinate boundary conditions [as documented in the corresponding NRPy+ tutorial module](Tutorial-Start_to_Finish-Curvilinear_BCs.ipynb)

In [11]:
import CurviBoundaryConditions_opt.CurviBoundaryConditions_opt as cbcs
cmd.mkdir(os.path.join(outdir,"boundary_conditions/"))
for file in ["apply_bcs_curvilinear.h","BCs_data_structs.h","bcstruct_freemem.h","CurviBC_include_Cfunctions.h",
             "driver_bcstruct.h","set_bcstruct.h","set_up__bc_gz_map_and_parity_condns.h"]:
    shutil.copy(os.path.join("CurviBoundaryConditions_opt","boundary_conditions", file), 
                os.path.join(outdir,"boundary_conditions/"))
cbcs.Set_up_CurviBoundaryConditions(os.path.join(outdir,"boundary_conditions/"))

Wrote to file "BSSN_Two_BHs_Collide-Psi4_Ccodes/boundary_conditions/parity_conditions_symbolic_dot_products.h"
Evolved gridfunction "aDD00" has parity type 4.
Evolved gridfunction "aDD01" has parity type 5.
Evolved gridfunction "aDD02" has parity type 6.
Evolved gridfunction "aDD11" has parity type 7.
Evolved gridfunction "aDD12" has parity type 8.
Evolved gridfunction "aDD22" has parity type 9.
Evolved gridfunction "alpha" has parity type 0.
Evolved gridfunction "betU0" has parity type 1.
Evolved gridfunction "betU1" has parity type 2.
Evolved gridfunction "betU2" has parity type 3.
Evolved gridfunction "cf" has parity type 0.
Evolved gridfunction "hDD00" has parity type 4.
Evolved gridfunction "hDD01" has parity type 5.
Evolved gridfunction "hDD02" has parity type 6.
Evolved gridfunction "hDD11" has parity type 7.
Evolved gridfunction "hDD12" has parity type 8.
Evolved gridfunction "hDD22" has parity type 9.
Evolved gridfunction "lambdaU0" has parity type 1.
Evolved gridfunction "lam

<a id='mainc'></a>

# Step 5: `BrillLindquist_Playground.c`: The Main C Code \[Back to [top](#toc)\]
$$\label{mainc}$$

In [12]:
# Part P0: Define REAL, set the number of ghost cells NGHOSTS (from NRPy+'s FD_CENTDERIVS_ORDER),
#          and set the CFL_FACTOR (which can be overwritten at the command line)

with open(outdir+"BSSN_Playground_REAL__NGHOSTS__CFL_FACTOR.h", "w") as file:
    file.write("""
// Part P0.a: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
#define NGHOSTS """+str(int(FD_order/2)+1)+"""
// Part P0.b: Set the numerical precision (REAL) to double, ensuring all floating point
//            numbers are stored to at least ~16 significant digits
#define REAL """+REAL+"""
// Part P0.c: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
REAL CFL_FACTOR = """+str(CFL_FACTOR)+"""; // Set the CFL Factor. Can be overwritten at command line.""")

In [13]:
%%writefile $outdir/BrillLindquist_Playground.c

// Step P0: Define REAL and NGHOSTS; and declare CFL_FACTOR. This header is generated in NRPy+.
#include "BSSN_Playground_REAL__NGHOSTS__CFL_FACTOR.h"

#include "rfm_files/rfm_struct__declare.h"

#include "declare_Cparameters_struct.h"

// All SIMD intrinsics used in SIMD-enabled C code loops are defined here:
#include "SIMD/SIMD_intrinsics.h"

// Step P1: Import needed header files
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "time.h"
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#ifndef M_PI
#define M_PI 3.141592653589793238462643383279502884L
#endif
#ifndef M_SQRT1_2
#define M_SQRT1_2 0.707106781186547524400844362104849039L
#endif
#define wavespeed 1.0 // Set CFL-based "wavespeed" to 1.0.

// Step P2: Declare the IDX4S(gf,i,j,k) macro, which enables us to store 4-dimensions of
//           data in a 1D array. In this case, consecutive values of "i" 
//           (all other indices held to a fixed value) are consecutive in memory, where 
//           consecutive values of "j" (fixing all other indices) are separated by 
//           Nxx_plus_2NGHOSTS0 elements in memory. Similarly, consecutive values of
//           "k" are separated by Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1 in memory, etc.
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )
#define IDX4ptS(g,idx) ( (idx) + (Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2) * (g) )
#define IDX3S(i,j,k) ( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) ) ) )
#define LOOP_REGION(i0min,i0max, i1min,i1max, i2min,i2max) \
  for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++)
#define LOOP_ALL_GFS_GPS(ii) _Pragma("omp parallel for") \
  for(int (ii)=0;(ii)<Nxx_plus_2NGHOSTS_tot*NUM_EVOL_GFS;(ii)++)

// Step P3: Set UUGF and VVGF macros, as well as xxCart()
#include "boundary_conditions/gridfunction_defines.h"

// Step P4: Set xxCart(const paramstruct *restrict params, 
//                     REAL *restrict xx[3],
//                     const int i0,const int i1,const int i2, 
//                     REAL xCart[3]),
//           which maps xx->Cartesian via
//    {xx[0][i0],xx[1][i1],xx[2][i2]}->{xCart[0],xCart[1],xCart[2]}
#include "xxCart.h"

// Step P5: Defines set_Nxx_dxx_invdx_params__and__xx(const int EigenCoord, const int Nxx[3], 
//                                       paramstruct *restrict params, REAL *restrict xx[3]),
//          which sets params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for
//          the chosen Eigen-CoordSystem if EigenCoord==1, or
//          CoordSystem if EigenCoord==0.
#include "set_Nxx_dxx_invdx_params__and__xx.h"

// Step P6: Include basic functions needed to impose curvilinear
//          parity and boundary conditions.
#include "boundary_conditions/CurviBC_include_Cfunctions.h"
    
// Step P7: Implement the algorithm for upwinding.
//          *NOTE*: This upwinding is backwards from
//          usual upwinding algorithms, because the
//          upwinding control vector in BSSN (the shift)
//          acts like a *negative* velocity.
//#define UPWIND_ALG(UpwindVecU) UpwindVecU > 0.0 ? 1.0 : 0.0

// Step P8: Include function for enforcing detgammabar constraint.
#include "enforce_detgammabar_constraint.h"

// Step P9: Find the CFL-constrained timestep
REAL find_timestep(const paramstruct *restrict params, REAL *restrict xx[3]) {
#include "set_Cparameters.h"
    REAL dsmin = 1e38; // Start with a crazy high value... close to the largest number in single precision.
    LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS0-NGHOSTS, 
                NGHOSTS,Nxx_plus_2NGHOSTS1-NGHOSTS, 
                NGHOSTS,Nxx_plus_2NGHOSTS2-NGHOSTS) {
        const REAL xx0 = xx[0][i0], xx1 = xx[1][i1], xx2 = xx[2][i2];
        REAL ds_dirn0, ds_dirn1, ds_dirn2;
        #include "ds_dirn.h"
        #ifndef MIN
        #define MIN(A, B) ( ((A) < (B)) ? (A) : (B) )
        #endif
        // Set dsmin = MIN(dsmin, ds_dirn0, ds_dirn1, ds_dirn2);
        dsmin = MIN(dsmin,MIN(ds_dirn0,MIN(ds_dirn1,ds_dirn2)));
    }
    return dsmin*CFL_FACTOR/wavespeed;
}

// Step P10: Declare function necessary for setting up the initial data.
// Step P10.a: Define BSSN_ID() for BrillLindquist initial data
#include "BrillLindquist.h"

// Step P10.b: Set the generic driver function for setting up BSSN initial data
void initial_data(const paramstruct *restrict params,REAL *restrict xx[3], REAL *restrict in_gfs) {
#include "set_Cparameters.h"
#pragma omp parallel for
  LOOP_REGION(0,Nxx_plus_2NGHOSTS0, 0,Nxx_plus_2NGHOSTS1, 0,Nxx_plus_2NGHOSTS2) {
    const int idx = IDX3S(i0,i1,i2);
    BSSN_ID(params,xx[0][i0],xx[1][i1],xx[2][i2],
            &in_gfs[IDX4ptS(HDD00GF,idx)],&in_gfs[IDX4ptS(HDD01GF,idx)],&in_gfs[IDX4ptS(HDD02GF,idx)],
            &in_gfs[IDX4ptS(HDD11GF,idx)],&in_gfs[IDX4ptS(HDD12GF,idx)],&in_gfs[IDX4ptS(HDD22GF,idx)],
            &in_gfs[IDX4ptS(ADD00GF,idx)],&in_gfs[IDX4ptS(ADD01GF,idx)],&in_gfs[IDX4ptS(ADD02GF,idx)],
            &in_gfs[IDX4ptS(ADD11GF,idx)],&in_gfs[IDX4ptS(ADD12GF,idx)],&in_gfs[IDX4ptS(ADD22GF,idx)],
            &in_gfs[IDX4ptS(TRKGF,idx)],
            &in_gfs[IDX4ptS(LAMBDAU0GF,idx)],&in_gfs[IDX4ptS(LAMBDAU1GF,idx)],&in_gfs[IDX4ptS(LAMBDAU2GF,idx)],
            &in_gfs[IDX4ptS(VETU0GF,idx)],&in_gfs[IDX4ptS(VETU1GF,idx)],&in_gfs[IDX4ptS(VETU2GF,idx)],
            &in_gfs[IDX4ptS(BETU0GF,idx)],&in_gfs[IDX4ptS(BETU1GF,idx)],&in_gfs[IDX4ptS(BETU2GF,idx)],
            &in_gfs[IDX4ptS(ALPHAGF,idx)],&in_gfs[IDX4ptS(CFGF,idx)]);
  }
}

// Step P11: Declare function for evaluating Hamiltonian constraint (diagnostic)
#include "Hamiltonian.h"    

// Step P12: Declare rhs_eval function, which evaluates BSSN RHSs
void rhs_eval(rfm_struct *restrict rfmstruct,
              const paramstruct *restrict params, const REAL *restrict auxevol_gfs,
              const REAL *restrict in_gfs,REAL *restrict rhs_gfs) {
#include "set_Cparameters-SIMD.h"
#include "BSSN_RHSs.h"
}

// Step P13: Declare Ricci_eval function, which evaluates Ricci tensor
void Ricci_eval(rfm_struct *restrict rfmstruct,
                const paramstruct *restrict params, 
                const REAL *restrict in_gfs,REAL *restrict auxevol_gfs) {
#include "set_Cparameters-SIMD.h"
#include "BSSN_Ricci.h"
}

// Step P14: Declare function for evaluating real and imaginary parts of psi4 (diagnostic)
void psi4(const paramstruct *restrict params,
          const int i0,const int i1,const int i2, 
          REAL *xx[3], 
          REAL *in_gfs, REAL *aux_gfs) {
#include "set_Cparameters.h"
    const int idx = IDX3S(i0,i1,i2);
    const REAL xx0 = xx[0][i0];
    const REAL xx1 = xx[1][i1];
    const REAL xx2 = xx[2][i2];
    //    REAL psi4_re_pt0,psi4_re_pt1,psi4_re_pt2;
    {
#include "Psi4re_pt0_lowlevel.h"
    }
    {
#include "Psi4re_pt1_lowlevel.h"
    }
    {
#include "Psi4re_pt2_lowlevel.h"
    }
//    REAL psi4_im_pt0,psi4_im_pt1,psi4_im_pt2;
    {
#include "Psi4im_pt0_lowlevel.h"
    }
    {
#include "Psi4im_pt1_lowlevel.h"
    }
    {
#include "Psi4im_pt2_lowlevel.h"
    }
    
//    aux_gfs[IDX4pt(PSI4RGF,idx)] = psi4_re_pt0 + psi4_re_pt1 + psi4_re_pt2;
//    aux_gfs[IDX4pt(PSI4IGF,idx)] = psi4_im_pt0 + psi4_im_pt1 + psi4_im_pt2;
}

// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Set up initial data to an exact solution
// Step 2: Start the timer, for keeping track of how fast the simulation is progressing.
// Step 3: Integrate the initial data forward in time using the chosen RK-like Method of 
//         Lines timestepping algorithm, and output periodic simulation diagnostics 
// Step 3.a: Output 2D data file periodically, for visualization
// Step 3.b: Step forward one timestep (t -> t+dt) in time using 
//           chosen RK-like MoL timestepping algorithm
// Step 3.c: If t=t_final, output conformal factor & Hamiltonian 
//           constraint violation to 2D data file
// Step 3.d: Progress indicator printing to stderr
// Step 4: Free all allocated memory
int main(int argc, const char *argv[]) {
    paramstruct params;
#include "set_Cparameters_default.h"
    
    // Step 0a: Read command-line input, error out if nonconformant
    if((argc != 4 && argc != 5) || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < 2 /* FIXME; allow for axisymmetric sims */) {
        fprintf(stderr,"Error: Expected three command-line arguments: ./BrillLindquist_Playground Nx0 Nx1 Nx2,\n");
        fprintf(stderr,"where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
        fprintf(stderr,"Nx[] MUST BE larger than NGHOSTS (= %d)\n",NGHOSTS);
        exit(1);
    }
    if(argc == 5) {
        CFL_FACTOR = strtod(argv[4],NULL);
        if(CFL_FACTOR > 0.5 && atoi(argv[3])!=2) {
            fprintf(stderr,"WARNING: CFL_FACTOR was set to %e, which is > 0.5.\n",CFL_FACTOR);
            fprintf(stderr,"         This will generally only be stable if the simulation is purely axisymmetric\n");
            fprintf(stderr,"         However, Nx2 was set to %d>2, which implies a non-axisymmetric simulation\n",atoi(argv[3]));
        }
    }
    // Step 0b: Set up numerical grid structure, first in space...
    const int Nxx[3] = { atoi(argv[1]), atoi(argv[2]), atoi(argv[3]) };
    if(Nxx[0]%2 != 0 || Nxx[1]%2 != 0 || Nxx[2]%2 != 0) {
        fprintf(stderr,"Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
        fprintf(stderr,"       For example, in case of angular directions, proper symmetry zones will not exist.\n");
        exit(1);
    }

    // Step 0c: Set free parameters, overwriting Cparameters defaults 
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"

   // Step 0d: Uniform coordinate grids are stored to *xx[3]
    REAL *xx[3];
    // Step 0d.i: Set bcstruct
    bc_struct bcstruct;
    {
        int EigenCoord = 1;
        // Step 0d.ii: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
        //             params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
        //             chosen Eigen-CoordSystem.
        set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);
        // Step 0d.iii: Set Nxx_plus_2NGHOSTS_tot
#include "set_Cparameters-nopointer.h"
        const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;
        // Step 0e: Find ghostzone mappings; set up bcstruct
#include "boundary_conditions/driver_bcstruct.h"
        // Step 0e.i: Free allocated space for xx[][] array
        for(int i=0;i<3;i++) free(xx[i]);
    }
    
    // Step 0f: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    //          params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    //          chosen (non-Eigen) CoordSystem.
    int EigenCoord = 0;
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);

    // Step 0g: Set all C parameters "blah" for params.blah, including
    //          Nxx_plus_2NGHOSTS0 = params.Nxx_plus_2NGHOSTS0, etc.
#include "set_Cparameters-nopointer.h"
    const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

    // Step 0h: Time coordinate parameters
    const REAL t_final =  275.0/300.0*domain_size; /* Final time is set so that at t=t_final, 
                                                    * data at the origin have not been corrupted 
                                                    * by the approximate outer boundary condition */
    
    // Step 0i: Set timestep based on smallest proper distance between gridpoints and CFL factor 
    REAL dt = find_timestep(&params, xx);
    //fprintf(stderr,"# Timestep set to = %e\n",(double)dt);
    int N_final = (int)(t_final / dt + 0.5); // The number of points in time. 
                                             // Add 0.5 to account for C rounding down 
                                             // typecasts to integers.
    REAL out_approx_every_t = 0.2;
    int output_every_N = (int)(out_approx_every_t*((REAL)N_final)/t_final);

    // Step 0j: Error out if the number of auxiliary gridfunctions outnumber evolved gridfunctions.
    //              This is a limitation of the RK method. You are always welcome to declare & allocate 
    //              additional gridfunctions by hand.
    if(NUM_AUX_GFS > NUM_EVOL_GFS) {
        fprintf(stderr,"Error: NUM_AUX_GFS > NUM_EVOL_GFS. Either reduce the number of auxiliary gridfunctions,\n");
        fprintf(stderr,"       or allocate (malloc) by hand storage for *diagnostic_output_gfs. \n");
        exit(1);
    }
    
    // Step 0k: Allocate memory for gridfunctions
#include "MoLtimestepping/RK_Allocate_Memory.h"
    REAL *restrict auxevol_gfs = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS_tot);
    
    // Step 0l: Set up precomputed referenc metric arrays
    // Step 0l.i: Allocate space for precomputed reference metric arrays.
#include "rfm_files/rfm_struct__malloc.h"

    // Step 0l.ii: Define precomputed reference metric arrays.
    {
    #include "set_Cparameters-nopointer.h"
    #include "rfm_files/rfm_struct__define.h"
    }

    // Step 1: Set up initial data to an exact solution
    initial_data(&params, xx, y_n_gfs);

    // Step 1b: Apply boundary conditions, as initial data 
    //          are sometimes ill-defined in ghost zones.
    //          E.g., spherical initial data might not be
    //          properly defined at points where r=-1.
    apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS,evol_gf_parity, y_n_gfs);
    enforce_detgammabar_constraint(&rfmstruct, &params, y_n_gfs);

    // Step 2: Start the timer, for keeping track of how fast the simulation is progressing.
#ifdef __linux__ // Use high-precision timer in Linux.
    struct timespec start, end;
    clock_gettime(CLOCK_REALTIME, &start);
#else     // Resort to low-resolution, standards-compliant timer in non-Linux OSs
    // http://www.cplusplus.com/reference/ctime/time/
    time_t start_timer,end_timer;
    time(&start_timer); // Resolution of one second...
#endif
    
    // Step 3: Integrate the initial data forward in time using the chosen RK-like Method of 
    //         Lines timestepping algorithm, and output periodic simulation diagnostics 
    for(int n=0;n<=N_final;n++) { // Main loop to progress forward in time.

        // Step 3.a: Output diagnostics
        if(n%output_every_N == 0) {
            // Step 3.a.i: Output psi4 spin-weight -2 decomposed data, every N_output_every */
#include "SpinWeight_minus2_SphHarmonics/SpinWeight_minus2_SphHarmonics.h"
            char filename[100];

            for(int r_ext_idx = (Nxx_plus_2NGHOSTS0-NGHOSTS)/4; 
                r_ext_idx<(Nxx_plus_2NGHOSTS0-NGHOSTS)*0.9;
                r_ext_idx+=5) {
                REAL r_ext;
                {
                    REAL xx0 = xx[0][r_ext_idx];
                    REAL xx1 = xx[1][1];
                    REAL xx2 = xx[2][1];
                    REAL xCart[3];
                    xxCart(&params,xx,r_ext_idx,1,1,xCart);
                    r_ext = sqrt(xCart[0]*xCart[0] + xCart[1]*xCart[1] + xCart[2]*xCart[2]);
                }

                sprintf(filename,"outPsi4_l2m0-%d-r%.2f.txt",Nxx[0],(double)r_ext);
                FILE *outPsi4_l2m0;
                if(n==0) outPsi4_l2m0 = fopen(filename, "w");
                else     outPsi4_l2m0 = fopen(filename, "a");
                REAL Psi4r_0pt_l2m0 = 0.0,Psi4r_1pt_l2m0 = 0.0,Psi4r_2pt_l2m0 = 0.0;
                REAL Psi4i_0pt_l2m0 = 0.0,Psi4i_1pt_l2m0 = 0.0,Psi4i_2pt_l2m0 = 0.0;
                LOOP_REGION(r_ext_idx,r_ext_idx+1, 
                            NGHOSTS,Nxx_plus_2NGHOSTS1-NGHOSTS, 
                            NGHOSTS,NGHOSTS+1) {
                    psi4(&params, i0,i1,i2,  xx,  y_n_gfs,  diagnostic_output_gfs);
                    const int idx = IDX3S(i0,i1,i2);
                    const REAL th = xx[1][i1];
                    const REAL ph = xx[2][i2];
                    // Construct integrand for Psi4 spin-weight s=-2,l=2,m=0 spherical harmonic

                    // Based on http://www.demonstrations.wolfram.com/SpinWeightedSphericalHarmonics/
                    // we have {}_{s}_Y_{lm} = {}_{-2}_Y_{20} = 1/4 * sqrt(15 / (2*pi)) * sin(th)^2
                    // Confirm integrand is correct:
                    // Integrate[(1/4 Sqrt[15/(2 \[Pi])] Sin[th]^2) (1/4 Sqrt[15/(2 \[Pi])] Sin[th]^2)*2*Pi*Sin[th], {th, 0, Pi}]
                    // ^^^ equals 1.
                    REAL ReY_sm2_l2_m0,ImY_sm2_l2_m0;
                    SpinWeight_minus2_SphHarmonics(2,0, th,ph,  &ReY_sm2_l2_m0,&ImY_sm2_l2_m0);
                    const REAL sinth = sin(xx[1][i1]);
                    /*                     psi4                       *{}_{-2}_Y_{20}* (int dphi)* sinth*dtheta */
                    Psi4r_0pt_l2m0 += diagnostic_output_gfs[IDX4ptS(PSI4R_0PTGF,idx)]*ReY_sm2_l2_m0 * (2*M_PI)  * sinth*dxx1;
                    Psi4r_1pt_l2m0 += diagnostic_output_gfs[IDX4ptS(PSI4R_1PTGF,idx)]*ReY_sm2_l2_m0 * (2*M_PI)  * sinth*dxx1;
                    Psi4r_2pt_l2m0 += diagnostic_output_gfs[IDX4ptS(PSI4R_2PTGF,idx)]*ReY_sm2_l2_m0 * (2*M_PI)  * sinth*dxx1;
                    Psi4i_0pt_l2m0 += diagnostic_output_gfs[IDX4ptS(PSI4I_0PTGF,idx)]*ImY_sm2_l2_m0 * (2*M_PI)  * sinth*dxx1;
                    Psi4i_1pt_l2m0 += diagnostic_output_gfs[IDX4ptS(PSI4I_1PTGF,idx)]*ImY_sm2_l2_m0 * (2*M_PI)  * sinth*dxx1;
                    Psi4i_2pt_l2m0 += diagnostic_output_gfs[IDX4ptS(PSI4I_2PTGF,idx)]*ImY_sm2_l2_m0 * (2*M_PI)  * sinth*dxx1;
                }
                fprintf(outPsi4_l2m0,"%e %.15e %.15e %.15e %.15e %.15e %.15e\n", (double)((n)*dt), 
                        (double)Psi4r_0pt_l2m0,(double)Psi4r_1pt_l2m0,(double)Psi4r_2pt_l2m0,
                        (double)Psi4i_0pt_l2m0,(double)Psi4i_1pt_l2m0,(double)Psi4i_2pt_l2m0);
                fclose(outPsi4_l2m0);
            }
        }

        // Step 3.b: Step forward one timestep (t -> t+dt) in time using 
        //           chosen RK-like MoL timestepping algorithm
#include "MoLtimestepping/RK_MoL.h"

        // Step 3.c: If t=t_final, output conformal factor & Hamiltonian 
        //           constraint violation to 2D data file
        if(n==N_final-1) {
            // Evaluate Hamiltonian constraint violation
            Hamiltonian_constraint(&params, xx, y_n_gfs, diagnostic_output_gfs);
            char filename[100];
            sprintf(filename,"out%d.txt",Nxx[0]);
            FILE *out2D = fopen(filename, "w");
            const int i0MIN=NGHOSTS; // In spherical, r=Delta r/2.
            const int i1mid=Nxx_plus_2NGHOSTS1/2;
            const int i2mid=Nxx_plus_2NGHOSTS2/2;
            LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS0-NGHOSTS, 
                        NGHOSTS,Nxx_plus_2NGHOSTS1-NGHOSTS, 
                        NGHOSTS,Nxx_plus_2NGHOSTS2-NGHOSTS) {
                REAL xx0 = xx[0][i0];
                REAL xx1 = xx[1][i1];
                REAL xx2 = xx[2][i2];
                REAL xCart[3];
                xxCart(&params,xx,i0,i1,i2,xCart);
                int idx = IDX3S(i0,i1,i2);
                fprintf(out2D,"%e %e %e %e\n",xCart[1],xCart[2], y_n_gfs[IDX4ptS(CFGF,idx)],
                        log10(fabs(diagnostic_output_gfs[IDX4ptS(HGF,idx)])));
            }
            fclose(out2D);
        }
        // Step 3.d: Progress indicator printing to stderr

        // Step 3.d.i: Measure average time per iteration
#ifdef __linux__ // Use high-precision timer in Linux.
        clock_gettime(CLOCK_REALTIME, &end);
        const long long unsigned int time_in_ns = 1000000000L * (end.tv_sec - start.tv_sec) + end.tv_nsec - start.tv_nsec;
#elif _WIN32     // Resort to low-resolution, standards-compliant timer in Windows
        time(&end_timer); // Resolution of one second...
        REAL time_in_ns = difftime(end_timer,start_timer)*1.0e9+0.5; // Round up to avoid divide-by-zero.
#endif
        const REAL s_per_iteration_avg = ((REAL)time_in_ns / (REAL)n) / 1.0e9;

        const int iterations_remaining = N_final - n;
        const REAL time_remaining_in_mins = s_per_iteration_avg * (REAL)iterations_remaining / 60.0;

        const REAL num_RHS_pt_evals = (REAL)(Nxx[0]*Nxx[1]*Nxx[2]) * 4.0 * (REAL)n; // 4 RHS evals per gridpoint for RK4
        const REAL RHS_pt_evals_per_sec = num_RHS_pt_evals / ((REAL)time_in_ns / 1.0e9);

        // Step 3.d.ii: Output simulation progress to stderr
        if(n % 10 == 0) {
            fprintf(stderr,"%c[2K", 27); // Clear the line
            fprintf(stderr,"It: %d t=%.2f dt=%.2e | %.1f%%; ETA %.0f s | t/h %.2f | gp/s %.2e\r",  // \r is carriage return, move cursor to the beginning of the line
                   n, n * (double)dt, (double)dt, (double)(100.0 * (REAL)n / (REAL)N_final),
                   (double)time_remaining_in_mins*60, (double)(dt * 3600.0 / s_per_iteration_avg), (double)RHS_pt_evals_per_sec);
            fflush(stderr); // Flush the stderr buffer
        } // End progress indicator if(n % 10 == 0)
    } // End main loop to progress forward in time.
    fprintf(stderr,"\n"); // Clear the final line of output from progress indicator.
    
    // Step 4: Free all allocated memory
#include "rfm_files/rfm_struct__freemem.h"
#include "boundary_conditions/bcstruct_freemem.h"
#include "MoLtimestepping/RK_Free_Memory.h"
    free(auxevol_gfs);
    for(int i=0;i<3;i++) free(xx[i]);

    return 0;
}

Writing BSSN_Two_BHs_Collide-Psi4_Ccodes//BrillLindquist_Playground.c


In [ ]:
import cmdline_helper as cmd
CFL_FACTOR=1.0
outdir="BSSN_Two_BHs_Collide_Ccodes/"
cmd.C_compile(outdir+"BrillLindquist_Playground.c", "BrillLindquist_Playground",compile_mode="optimized")
# cmd.C_compile(outdir+"BrillLindquist_Playground.c", "BrillLindquist_Playground",compile_mode="custom",
#              custom_compile_string="gcc -O2 -g -march=native BSSN_Two_BHs_Collide_Ccodes/BrillLindquist_Playground.c -o BrillLindquist_Playground -lm")

cmd.delete_existing_files("out*.txt")
cmd.delete_existing_files("out*.png")
# cmd.Execute("BrillLindquist_Playground", "72 12 2 "+str(CFL_FACTOR))
cmd.Execute("BrillLindquist_Playground", "800 16 2 "+str(CFL_FACTOR))

# with open("compilescript", "w") as file:
#     count=0
#     for custom_compile_string0 in ["-O2","-O",""]:
#         for custom_compile_string1 in ["","-fp-model fast=2 -no-prec-div"]:
#             for custom_compile_string2 in ["","-qopt-prefetch=3","-qopt-prefetch=4"]:
#                 for custom_compile_string3 in ["","-unroll"]:
#                     for custom_compile_string4 in ["","-qoverride-limits"]:
#                         exc= "BL"+custom_compile_string0+custom_compile_string1.replace(" ","")+custom_compile_string2+custom_compile_string3+custom_compile_string4
#                         ccs = "icc -qopenmp -xHost "+custom_compile_string0+" "+custom_compile_string1+" "+custom_compile_string2+" "+custom_compile_string3+" "+custom_compile_string4+" BSSN_Two_BHs_Collide_Ccodes/BrillLindquist_Playground.c -o "+exc
#                         file.write(ccs+" &\n")
#                         if count>0 and count%16==0:
#                             file.write("wait\n")
#                         count += 1
#     file.write("wait\n")

# with open("compilescriptgcc", "w") as file:
#     count=0
#     for custom_compile_string0 in ["-Ofast","-O2","-O3","-O",""]:
#         for custom_compile_string1 in ["-fopenmp"]:
#             for custom_compile_string2 in ["","-march=native"]:
#                 for custom_compile_string3 in ["","-funroll-loops","-funroll-all-loops"]:
#                     for custom_compile_string4 in [""]:
#                         exc= "BL"+custom_compile_string0+custom_compile_string1+custom_compile_string2+custom_compile_string3+custom_compile_string4
#                         ccs = "gcc "+custom_compile_string0+" "+custom_compile_string1+" "+custom_compile_string2+" "+custom_compile_string3+" "+custom_compile_string4+" BSSN_Two_BHs_Collide_Ccodes/BrillLindquist_Playground.c -o "+exc
#                         file.write(ccs+" -lm &\n")
#                         if count>0 and count%16==0:
#                             file.write("wait\n")
#                         count += 1
#     file.write("wait\n")

    
print("Finished this code cell.")

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops BSSN_Two_BHs_Collide_Ccodes/BrillLindquist_Playground.c -o BrillLindquist_Playground -lm`...
Finished executing in 24.893226862 seconds.
Finished compilation.
Executing `taskset -c 0,1,2,3 ./BrillLindquist_Playground 800 16 2 1.0`...


<a id='visualize'></a>

# Step 6: Data Visualization Animations \[Back to [top](#toc)\]
$$\label{visualize}$$ 

In this section we will generate a movie, plotting the conformal factor of these initial data on a 2D grid, such that darker colors imply stronger gravitational fields. Hence, we see the two black holes initially centered at $z/M=\pm 0.5$, where $M$ is an arbitrary mass scale (conventionally the [ADM mass](https://en.wikipedia.org/w/index.php?title=ADM_formalism&oldid=846335453) is chosen), and our formulation of Einstein's equations adopt $G=c=1$ [geometrized units](https://en.wikipedia.org/w/index.php?title=Geometrized_unit_system&oldid=861682626).

<a id='installdownload'></a>

## Step 6.a: Install `scipy` and download `ffmpeg` if they are not yet installed/downloaded \[Back to [top](#toc)\]
$$\label{installdownload}$$ 

Note that if you are not running this within `mybinder`, but on a Windows system, `ffmpeg` must be installed using a separate package (on [this site](http://ffmpeg.org/)), or if running Jupyter within Anaconda, use the command: `conda install -c conda-forge ffmpeg`.

In [ ]:
!pip install scipy > /dev/null

check_for_ffmpeg = !which ffmpeg >/dev/null && echo $?
if check_for_ffmpeg != ['0']:
    print("Couldn't find ffmpeg, so I'll download it.")
    # Courtesy https://johnvansickle.com/ffmpeg/
    !wget http://astro.phys.wvu.edu/zetienne/ffmpeg-static-amd64-johnvansickle.tar.xz
    !tar Jxf ffmpeg-static-amd64-johnvansickle.tar.xz
    print("Copying ffmpeg to ~/.local/bin/. Assumes ~/.local/bin is in the PATH.")
    !mkdir ~/.local/bin/
    !cp ffmpeg-static-amd64-johnvansickle/ffmpeg ~/.local/bin/
    print("If this doesn't work, then install ffmpeg yourself. It should work fine on mybinder.")

<a id='genimages'></a>

## Step 6.b: Generate images for visualization animation \[Back to [top](#toc)\]
$$\label{genimages}$$ 

Here we loop through the data files output by the executable compiled and run in [the previous step](#mainc), generating a [png](https://en.wikipedia.org/wiki/Portable_Network_Graphics) image for each data file.

**Special thanks to Terrence Pierre Jacques. His work with the first versions of these scripts greatly contributed to the scripts as they exist below.**

In [ ]:
# ## VISUALIZATION ANIMATION, PART 1: Generate PNGs, one per frame of movie ##

# import numpy as np
# from scipy.interpolate import griddata
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import savefig
# from IPython.display import HTML
# import matplotlib.image as mgimg

# import glob
# import sys
# from matplotlib import animation

# globby = glob.glob('out96-00*.txt')
# file_list = []
# for x in sorted(globby):
#     file_list.append(x)

# bound=1.4
# pl_xmin = -bound
# pl_xmax = +bound
# pl_ymin = -bound
# pl_ymax = +bound

# for filename in file_list:
#     fig = plt.figure()
#     x,y,cf,Ham = np.loadtxt(filename).T #Transposed for easier unpacking

#     plotquantity = cf
#     plotdescription = "Numerical Soln."
#     plt.title("Black Hole Head-on Collision (conf factor)")
#     plt.xlabel("y/M")
#     plt.ylabel("z/M")

#     grid_x, grid_y = np.mgrid[pl_xmin:pl_xmax:300j, pl_ymin:pl_ymax:300j]
#     points = np.zeros((len(x), 2))
#     for i in range(len(x)):
#         # Zach says: No idea why x and y get flipped...
#         points[i][0] = y[i]
#         points[i][1] = x[i]

#     grid = griddata(points, plotquantity, (grid_x, grid_y), method='nearest')
#     gridcub = griddata(points, plotquantity, (grid_x, grid_y), method='cubic')
#     im = plt.imshow(gridcub, extent=(pl_xmin,pl_xmax, pl_ymin,pl_ymax))
#     ax = plt.colorbar()
#     ax.set_label(plotdescription)
#     savefig(filename+".png",dpi=150)
#     plt.close(fig)
#     sys.stdout.write("%c[2K" % 27)
#     sys.stdout.write("Processing file "+filename+"\r")
#     sys.stdout.flush()

<a id='genvideo'></a>

## Step 6.c: Generate visualization animation \[Back to [top](#toc)\]
$$\label{genvideo}$$ 

In the following step, [ffmpeg](http://ffmpeg.org) is used to generate an [mp4](https://en.wikipedia.org/wiki/MPEG-4) video file, which can be played directly from this Jupyter notebook.

In [ ]:
# ## VISUALIZATION ANIMATION, PART 2: Combine PNGs to generate movie ##

# # https://stackoverflow.com/questions/14908576/how-to-remove-frame-from-matplotlib-pyplot-figure-vs-matplotlib-figure-frame
# # https://stackoverflow.com/questions/23176161/animating-pngs-in-matplotlib-using-artistanimation

# fig = plt.figure(frameon=False)
# ax = fig.add_axes([0, 0, 1, 1])
# ax.axis('off')

# myimages = []

# for i in range(len(file_list)):
#     img = mgimg.imread(file_list[i]+".png")
#     imgplot = plt.imshow(img)
#     myimages.append([imgplot])

# ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
# plt.close()
# ani.save('BH_Head-on_Collision.mp4', fps=5,dpi=150)

In [ ]:
## VISUALIZATION ANIMATION, PART 3: Display movie as embedded HTML5 (see next cell) ##

# https://stackoverflow.com/questions/18019477/how-can-i-play-a-local-video-in-my-ipython-notebook

In [ ]:
# %%HTML
# <video width="480" height="360" controls>
#   <source src="BH_Head-on_Collision.mp4" type="video/mp4">
# </video>

<a id='convergence'></a>

# Step 7: Visualize the numerical error, and confirm that it converges to zero with increasing numerical resolution (sampling) \[Back to [top](#toc)\]
$$\label{convergence}$$

In [ ]:
# x96,y96,valuesCF96,valuesHam96 = np.loadtxt('out96.txt').T #Transposed for easier unpacking

# pl_xmin = -2.5
# pl_xmax = +2.5
# pl_ymin = -2.5
# pl_ymax = +2.5

# grid_x, grid_y = np.mgrid[pl_xmin:pl_xmax:100j, pl_ymin:pl_ymax:100j]
# points96 = np.zeros((len(x96), 2))
# for i in range(len(x96)):
#     points96[i][0] = x96[i]
#     points96[i][1] = y96[i]

# grid96 = griddata(points96, valuesCF96, (grid_x, grid_y), method='nearest')
# grid96cub = griddata(points96, valuesCF96, (grid_x, grid_y), method='cubic')

# grid96 = griddata(points96, valuesHam96, (grid_x, grid_y), method='nearest')
# grid96cub = griddata(points96, valuesHam96, (grid_x, grid_y), method='cubic')

# # fig, ax = plt.subplots()

# plt.clf()
# plt.title("96x16 Num. Err.: log_{10}|Ham|")
# plt.xlabel("x/M")
# plt.ylabel("z/M")

# fig96cub = plt.imshow(grid96cub.T, extent=(pl_xmin,pl_xmax, pl_ymin,pl_ymax))
# cb = plt.colorbar(fig96cub)

In [ ]:
# x72,y72,valuesCF72,valuesHam72 = np.loadtxt('out72.txt').T #Transposed for easier unpacking
# points72 = np.zeros((len(x72), 2))
# for i in range(len(x72)):
#     points72[i][0] = x72[i]
#     points72[i][1] = y72[i]

# grid72 = griddata(points72, valuesHam72, (grid_x, grid_y), method='nearest')

# griddiff_72_minus_96 = np.zeros((100,100))
# griddiff_72_minus_96_1darray = np.zeros(100*100)
# gridx_1darray_yeq0 = np.zeros(100)
# grid72_1darray_yeq0 = np.zeros(100)
# grid96_1darray_yeq0 = np.zeros(100)
# count = 0
# for i in range(100):
#     for j in range(100):
#         griddiff_72_minus_96[i][j] = grid72[i][j] - grid96[i][j]
#         griddiff_72_minus_96_1darray[count] = griddiff_72_minus_96[i][j]
#         if j==49:
#             gridx_1darray_yeq0[i] = grid_x[i][j]
#             grid72_1darray_yeq0[i] = grid72[i][j] + np.log10((72./96.)**4)
#             grid96_1darray_yeq0[i] = grid96[i][j]
#         count = count + 1

# plt.clf()
# fig, ax = plt.subplots()
# plt.title("4th-order Convergence, at t/M=7.5 (post-merger; horiz at x/M=+/-1)")
# plt.xlabel("x/M")
# plt.ylabel("log10(Relative error)")

# ax.plot(gridx_1darray_yeq0, grid96_1darray_yeq0, 'k-', label='Nr=96')
# ax.plot(gridx_1darray_yeq0, grid72_1darray_yeq0, 'k--', label='Nr=72, mult by (72/96)^4')
# ax.set_ylim([-8.5,0.5])

# legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large')
# legend.get_frame().set_facecolor('C1')
# plt.show()

<a id='latex_pdf_output'></a>

# Step 8: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide.pdf](Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide-opt.ipynb
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide-opt.tex
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide-opt.tex
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide-opt.tex
!rm -f Tut*.out Tut*.aux Tut*.log